In [153]:
import pandas as pd
import numpy as np
import statistics

In [154]:
filename = pd.read_csv('/content/drive/MyDrive/PTDataScience/PandasForDataManipulation/sales_predictions.csv')
df = filename
df.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [155]:
df['Item_Fat_Content'].value_counts()#It looks like we will need to change LF reg and low fat to Low Fat and Regular

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [156]:
df.dtypes;
df.info()#it looks like there are some missing values in Item_Weight and Outlet_Size

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


# Outlet Size Solutions

In [157]:
df['Outlet_Size'].value_counts()# this indicates to me that there are a lot of 
                                #missing high values but its difficult to determine 
                                #how much of the NaN should be converted to High.
                                #I tried to see if there was a visable corralation 
                                #between the Outlet_Size value and any other columns and don't obviously see anything to connect the values to 

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [158]:
#df['Outlet_Identifier'].value_counts()#cool, it looks like outlet010 and 019 
#are missing some values and it seems like outlet size and id would be directly connected, 
#like out49 should always be medium
df.groupby('Outlet_Identifier')['Outlet_Size'].count() #shoot, I was hoping that 
                                                      #there would be at least one Outlet_size in 
                                                      #each of the IDs so that I could use that value to fill in the rest

Outlet_Identifier
OUT010      0
OUT013    932
OUT017      0
OUT018    928
OUT019    528
OUT027    935
OUT035    930
OUT045      0
OUT046    930
OUT049    930
Name: Outlet_Size, dtype: int64

In [159]:
df.groupby('Outlet_Location_Type')['Outlet_Size'].value_counts()#beautiful, so it looks like 
                                                                #Tier 1 historically have been only Small/Medium
                                                                #Tier 2 historically have been only Small
                                                                #Tier 3 historically have been Medium/ High

Outlet_Location_Type  Outlet_Size
Tier 1                Small          1458
                      Medium          930
Tier 2                Small           930
Tier 3                Medium         1863
                      High            932
Name: Outlet_Size, dtype: int64

In [160]:
df.groupby('Outlet_Identifier')['Outlet_Location_Type'].value_counts()#yikes, so OUT010 could be med/high.
                                                                      #but look, OUT 17 and 45 is Tier 2! Therefore, historically small

Outlet_Identifier  Outlet_Location_Type
OUT010             Tier 3                  555
OUT013             Tier 3                  932
OUT017             Tier 2                  926
OUT018             Tier 3                  928
OUT019             Tier 1                  528
OUT027             Tier 3                  935
OUT035             Tier 2                  930
OUT045             Tier 2                  929
OUT046             Tier 1                  930
OUT049             Tier 1                  930
Name: Outlet_Location_Type, dtype: int64

In [161]:
#maybe there is a year in which outlets were building a majority of s/m/h store. *This is Cass's idea when she was helping me think through this*
df.groupby('Outlet_Establishment_Year')['Outlet_Size'].value_counts() #yo, that's wicked! Woot woot, lets hope that OUT10,17,45 weren't built in 1985

Outlet_Establishment_Year  Outlet_Size
1985                       Medium         935
                           Small          528
1987                       High           932
1997                       Small          930
1999                       Medium         930
2004                       Small          930
2009                       Medium         928
Name: Outlet_Size, dtype: int64

In [162]:
df.groupby('Outlet_Identifier')['Outlet_Establishment_Year'].value_counts()#well they weren't. They were built in years that ain't got no Outlet_Size data, cool.

Outlet_Identifier  Outlet_Establishment_Year
OUT010             1998                         555
OUT013             1987                         932
OUT017             2007                         926
OUT018             2009                         928
OUT019             1985                         528
OUT027             1985                         935
OUT035             2004                         930
OUT045             2002                         929
OUT046             1997                         930
OUT049             1999                         930
Name: Outlet_Establishment_Year, dtype: int64

In [163]:
#before I fill in the missing values I should add another column to flag that the values were missing. 
#That way if my data coefficent is super off for flagged rows I will know that I guessed wrong when filling in those rows.

In [164]:
df['Flagged_Item_Weight'] = df.Item_Weight.map({np.NaN: 1}).fillna(0) #this wasn't working for the longest until we added np. Thanks to Cass and Nikitha 
df.loc[0:10,['Flagged_Item_Weight','Item_Weight']]#great, so now we have a system to look back on if the data is messed up in the future.

,Flagged_Item_Weight,Item_Weight
0,0.0,9.300
1,0.0,5.920
2,0.0,17.500
3,0.0,19.200
4,0.0,8.930
5,0.0,10.395
6,0.0,13.650
7,1.0,NaN
8,0.0,16.200
9,0.0,19.200


In [165]:
df['Flagged_Outlet_Size'] = df.Outlet_Size.map({np.NaN: 1}).fillna(0)
df.loc[0:10,['Flagged_Outlet_Size','Outlet_Size']];

Filter_17 = df['Outlet_Identifier'] == 'OUT017'
Filter_45 = df['Outlet_Identifier'] =='OUT045'

#NV = df[Filter_17]['Outlet_Size'].fillna("Small") #df.loc[Filter_17 & Filter_45, :]
df.loc[Filter_17, 'Outlet_Size'] = df.loc[Filter_17, 'Outlet_Size'].fillna('Small')
df.loc[Filter_45, 'Outlet_Size'] = df.loc[Filter_45, 'Outlet_Size'].fillna('Small')

In [166]:
df.info()
8523-7968

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                7968 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  Flagged_Item_Weight        8523 non-null   float64
 13  Flagged_Outlet_Size        8523 non-null   float

555

In [167]:
df[Filter_17].head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Flagged_Item_Weight,Flagged_Outlet_Size
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Small,Tier 2,Supermarket Type1,4710.5350,0.0,1.0
25,NCD06,13.000,Low Fat,0.099887,Household,45.9060,OUT017,2007,Small,Tier 2,Supermarket Type1,838.9080,0.0,1.0
53,FDA43,10.895,Low Fat,0.065042,Fruits and Vegetables,196.3794,OUT017,2007,Small,Tier 2,Supermarket Type1,3121.2704,0.0,1.0
54,NCP18,12.150,Low Fat,0.028760,Household,151.4708,OUT017,2007,Small,Tier 2,Supermarket Type1,4815.0656,0.0,1.0
73,FDG02,7.855,Low Fat,0.011325,Canned,189.6188,OUT017,2007,Small,Tier 2,Supermarket Type1,2285.0256,0.0,1.0


In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                7968 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  Flagged_Item_Weight        8523 non-null   float64
 13  Flagged_Outlet_Size        8523 non-null   float

In [169]:
# so now I need to make a decision for what I'm going to do with OUT10
Filter_010= df['Outlet_Identifier']== "OUT010"
#df[Filter_]

In [170]:
df.groupby('Outlet_Establishment_Year')['Outlet_Size'].value_counts() #I'm going to go with medium. 
#Its Tier 3 so its either that or high and the only time in this outlets hisotry that they built a 
#"high" store was in 1987 and since then it looks like they've gone with mostly with small or medium stores 


Outlet_Establishment_Year  Outlet_Size
1985                       Medium         935
                           Small          528
1987                       High           932
1997                       Small          930
1999                       Medium         930
2002                       Small          929
2004                       Small          930
2007                       Small          926
2009                       Medium         928
Name: Outlet_Size, dtype: int64

In [171]:
df.loc[Filter_010, 'Outlet_Size'] = df.loc[Filter_010, 'Outlet_Size'].map({np.NaN:'Medium'})
df[Filter_010]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Flagged_Item_Weight,Flagged_Outlet_Size
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.3800,0.0,1.0
28,FDE51,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,Medium,Tier 3,Grocery Store,178.4344,0.0,1.0
30,FDV38,19.250,Low Fat,0.170349,Dairy,55.7956,OUT010,1998,Medium,Tier 3,Grocery Store,163.7868,0.0,1.0
45,FDM39,6.420,LF,0.089499,Dairy,178.1002,OUT010,1998,Medium,Tier 3,Grocery Store,358.2004,0.0,1.0
65,FDC46,17.700,LF,0.195068,Snack Foods,185.4266,OUT010,1998,Medium,Tier 3,Grocery Store,184.4266,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8400,NCE07,8.180,Low Fat,0.021977,Household,140.6154,OUT010,1998,Medium,Tier 3,Grocery Store,425.4462,0.0,1.0
8432,FDI16,14.000,Regular,0.227261,Frozen Foods,54.3640,OUT010,1998,Medium,Tier 3,Grocery Store,159.7920,0.0,1.0
8473,DRI47,14.700,Low Fat,0.035016,Hard Drinks,144.3128,OUT010,1998,Medium,Tier 3,Grocery Store,431.4384,0.0,1.0
8486,FDR20,20.000,Regular,0.000000,Fruits and Vegetables,46.4744,OUT010,1998,Medium,Tier 3,Grocery Store,45.2744,0.0,1.0


#Item Weight Solutions

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  Flagged_Item_Weight        8523 non-null   float64
 13  Flagged_Outlet_Size        8523 non-null   float

Ok, so I bet that there are alot of objects that have an ID and Weight in most most columns except maybe one outlet. So I should be able to associate a weight with a ID and then apply that association to a whole store if an item has been used in any other store. But first I need to test that assumption by seeing how many outlets have null values in their Item Weight column

In [173]:
#df.groupby('Item_Identifier')['Item_Weight'].isnull().sum()
df.groupby('Outlet_Identifier')[ 'Item_Weight'].count()#so I was actually trying to just count null but this will work.

Outlet_Identifier
OUT010    555
OUT013    932
OUT017    926
OUT018    928
OUT019      0
OUT027      0
OUT035    930
OUT045    929
OUT046    930
OUT049    930
Name: Item_Weight, dtype: int64

Looks like that bet was correct

In [174]:
#df['Out_Identifier']['Item_Weight']
#Filter_019 = df['Outlet_Identifier'] =="OUT019"
search = df[df['Item_Identifier'].str.contains('FDC37', na=False)] #help from Andre Connor


I can't just manually go in and find IDs and fill in NaNs using ffill or bfill for a 1500 items. So I could...

1.   Make a dictionary using another stores ID and Item Weight. Then import that library on my Outlet 019 and Outlet 27 untill I get 0 NaNs


*   Might take several iterations
*   I've never made a dic from a df...


2.   Make a loop for any items with NaN to search for the items IDs and then fill with the mode of the results. 


*   Clean and one go once made correctly. 
*   Might be above my skills


Then go back and do a value_counts on the I_ID and make sure there are not mutiple values for each ID 


In [175]:
df.groupby('Item_Identifier').Item_Weight.apply(lambda x: x.median())# weird hack Cass found because .mode() doesn't work

Item_Identifier
DRA12    11.600
DRA24    19.350
DRA59     8.270
DRB01     7.390
DRB13     6.115
          ...  
NCZ30     6.590
NCZ41    19.850
NCZ42    10.500
NCZ53     9.600
NCZ54    14.650
Name: Item_Weight, Length: 1559, dtype: float64

Instead of the ideas I had I end up implimenting this awesome fillna idea with a .transform and medium instead of mode because they would end up being the same anyways in this case 

In [176]:
df['Item_Weight'] = df['Item_Weight'].fillna(df.groupby('Item_Identifier')['Item_Weight'].transform('median'))#https://stackoverflow.com/questions/40299055/pandas-how-to-fill-null-values-with-mean-of-a-groupby

In [177]:

df[(df['Flagged_Item_Weight'] == 0)& (df['Item_Identifier'] == 'NCZ53')]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Flagged_Item_Weight,Flagged_Outlet_Size
1267,NCZ53,9.6,Low Fat,0.024577,Health and Hygiene,186.6214,OUT018,2009,Medium,Tier 3,Supermarket Type2,1884.2140,0.0,0.0
3959,NCZ53,9.6,Low Fat,0.024457,Health and Hygiene,186.6214,OUT013,1987,High,Tier 3,Supermarket Type1,3203.1638,0.0,0.0
4924,NCZ53,9.6,Low Fat,0.040970,Health and Hygiene,188.7214,OUT010,1998,Medium,Tier 3,Grocery Store,376.8428,0.0,1.0
6879,NCZ53,9.6,Low Fat,0.024515,Health and Hygiene,189.2214,OUT049,1999,Medium,Tier 1,Supermarket Type1,3956.8494,0.0,0.0


In [178]:
df.info()#if df['Item_Weight'] == np.NaN : 
  #then #take Item_Identifier in that row 
       #find other rows with that Item_Identifier
       # take the Mode of the Item_Weight and fill the NaN with it
       

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  Flagged_Item_Weight        8523 non-null   float64
 13  Flagged_Outlet_Size        8523 non-null   float

In [179]:
df.dropna(inplace=True)

In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8519 entries, 0 to 8522
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8519 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8519 non-null   object 
 3   Item_Visibility            8519 non-null   float64
 4   Item_Type                  8519 non-null   object 
 5   Item_MRP                   8519 non-null   float64
 6   Outlet_Identifier          8519 non-null   object 
 7   Outlet_Establishment_Year  8519 non-null   int64  
 8   Outlet_Size                8519 non-null   object 
 9   Outlet_Location_Type       8519 non-null   object 
 10  Outlet_Type                8519 non-null   object 
 11  Item_Outlet_Sales          8519 non-null   float64
 12  Flagged_Item_Weight        8519 non-null   float64
 13  Flagged_Outlet_Size        8519 non-null   float

In [180]:
df[].value_counts()
#sort for numbers
#dtypes

# Item Fat Content Solution

In [189]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('LF', 'Low Fat') #LF reg and low fat to Low Fat and Regular
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('low fat', 'Low Fat')
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('reg', 'Regular')

In [190]:
df['Item_Fat_Content'].value_counts()

Low Fat    5516
Regular    3003
Name: Item_Fat_Content, dtype: int64